In [1]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/yoga/poses/all")

In [4]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [5]:
all_pose_links = soup.find(class_="nested-layout").find_all("a")

In [6]:
pose_links = ["https://www.tummee.com" + pose["href"] for pose in all_pose_links]

In [8]:
len(pose_links)

3639

In [7]:
def get_pose_info(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    try: 
        pose = soup.find("h1").text
        position = soup.find(class_="pose-summary-header", text="Position").find_next("td").text 
        pose_type = soup.find(class_="pose-summary-header", text="Type").find_next("td").text.split()
        base_pose = soup.find("main").find(class_="pose-section").find_next(class_="pose-section").find_next(class_="pose-section").find("a").text
        if base_pose == "submit request here":
            base_pose = soup.find("main").find(class_="pose-section").find_next(class_="pose-section").find_next(class_="pose-section").find_next(class_="pose-section").find("a").text

        if soup.find("span", class_="pose-summary-level-Beginner"):
            level = soup.find("span", class_="pose-summary-level-Beginner").text
        elif soup.find("span", class_="pose-summary-level-Intermediate"):
            level = soup.find("span", class_="pose-summary-level-Intermediate").text
        else:
            level = soup.find("span", class_="pose-summary-level-Advanced").text
        
        try:
            chakras = [i.text for i in soup.find(class_="pose-summary-header", text="Chakras").find_next("a").find_all("span")]
        except AttributeError:
            chakras = np.nan
            
    except AttributeError:
        info = {"Pose": np.nan}
    except TimeoutError:
        info = {"Pose": np.nan}
    else: 
        info = {"Pose": pose, "Level": level, "Position": position, "Movement Type": pose_type, "Chakras": chakras, "Base Pose": base_pose}
    return info

In [50]:
df3 = pd.DataFrame(names)

In [51]:
df3

,Pose,Pose Name
0,Complete Boat Pose,Complete Boat Pose
1,Boat Pose Aerial Variation,Boat Pose Aerial Variation
2,Boat Pose Aerial,Boat Pose Aerial
3,Boat Pose Variation 1 Arms Overhead,Boat Pose Variation 1 Arms Overhead
4,Boat Pose Arms Overhead,Boat Pose Arms Overhead
...,...,...
3634,Pawanmuktasana Tadasana Vinyasa,Wind Release Pose To Standing Pose Flow
3635,Supine Hip Flexor Roller,Supine Hip Flexor Roller
3636,Supine Knee Rolls Straps,Supine Knee Rolls Straps
3637,Supine Leg Lifts With Baby,Supine Leg Lifts With Baby


In [9]:
poses = [get_pose_info(url) for url in pose_links[0:2000]]

In [52]:
poses2 = [get_pose_info(url) for url in pose_links[2000:]]

In [45]:
df = pd.DataFrame(poses)

In [48]:
df2 = pd.DataFrame(poses2)

In [53]:
df = pd.concat([df, df2])

In [66]:
df.reset_index(inplace=True)
del df["index"]

In [67]:
df

,Pose,Level,Position,Movement Type,Chakras,Base Pose
0,Complete Boat Pose,Intermediate,Sitting,"[Forward-Bend,, Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Complete Boat Pose Contraindications
1,Boat Pose Aerial Variation,Intermediate,Sitting,"[Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Navasana
2,Boat Pose Aerial,Advanced,Sitting,"[Balance,, Strength]",[Solar Plexus (Manipura Chakra)],Boat Pose
3,Boat Pose Variation 1 Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
4,Boat Pose Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch]",[Solar Plexus (Manipura Chakra)],Boat Pose
...,...,...,...,...,...,...
3634,Pawanmuktasana Tadasana Vinyasa,Beginner,Supine,[Strength],[Solar Plexus (Manipura Chakra)],Wind Release Pose
3635,Supine Hip Flexor Roller,Intermediate,Supine,"[Strength,, Stretch,, Balance]",NaN,Wind Release Pose
3636,Supine Knee Rolls Straps,Beginner,Supine,[Stretch],NaN,Wind Release Pose
3637,Supine Leg Lifts With Baby,Beginner,Supine,[Strength],NaN,Wind Release Pose


Running into issues merging with other dataframe of class sequences because half of the pose names are listed in english here, and half in sanskrit. Going back to grab all english names so I can merge with the other dataframe of all english names: 

In [84]:
def get_english_names(url): # going back to grab these. didn't realize half were in sanskrit... and hence can't be merged with other dataframe
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    try: 
        pose = soup.find("h1").text
        english_name = soup.find(class_="pose-summary-header", text="English").find_next("td").text
    except AttributeError:
        info = {"Pose": np.nan}
    except TimeoutError:
        info = {"Pose": np.nan}
    else: 
        info = {"Pose": pose, "Pose Name": english_name}
    return info

In [85]:
names = [get_english_names(url) for url in pose_links]

In [86]:
df3 = pd.DataFrame(names)

In [87]:
df3

,Pose,Pose Name
0,Complete Boat Pose,Complete Boat Pose
1,Boat Pose Aerial Variation,Boat Pose Aerial Variation
2,Boat Pose Aerial,Boat Pose Aerial
3,Boat Pose Variation 1 Arms Overhead,Boat Pose Variation 1 Arms Overhead
4,Boat Pose Arms Overhead,Boat Pose Arms Overhead
...,...,...
3634,Pawanmuktasana Tadasana Vinyasa,Wind Release Pose To Standing Pose Flow
3635,Supine Hip Flexor Roller,Supine Hip Flexor Roller
3636,Supine Knee Rolls Straps,Supine Knee Rolls Straps
3637,Supine Leg Lifts With Baby,Supine Leg Lifts With Baby


Now combining the dataframes so they will have the english names, and dropping the other name columns with mixes of sanskrit and english: 

In [88]:
yoga_poses_df = pd.concat([df, df3], axis=1)

In [89]:
yoga_poses_df

,Pose,Level,Position,Movement Type,Chakras,Base Pose,Pose,Pose Name
0,Complete Boat Pose,Intermediate,Sitting,"[Forward-Bend,, Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Complete Boat Pose Contraindications,Complete Boat Pose,Complete Boat Pose
1,Boat Pose Aerial Variation,Intermediate,Sitting,"[Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Navasana,Boat Pose Aerial Variation,Boat Pose Aerial Variation
2,Boat Pose Aerial,Advanced,Sitting,"[Balance,, Strength]",[Solar Plexus (Manipura Chakra)],Boat Pose,Boat Pose Aerial,Boat Pose Aerial
3,Boat Pose Variation 1 Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose,Boat Pose Variation 1 Arms Overhead,Boat Pose Variation 1 Arms Overhead
4,Boat Pose Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch]",[Solar Plexus (Manipura Chakra)],Boat Pose,Boat Pose Arms Overhead,Boat Pose Arms Overhead
...,...,...,...,...,...,...,...,...
3634,Pawanmuktasana Tadasana Vinyasa,Beginner,Supine,[Strength],[Solar Plexus (Manipura Chakra)],Wind Release Pose,Pawanmuktasana Tadasana Vinyasa,Wind Release Pose To Standing Pose Flow
3635,Supine Hip Flexor Roller,Intermediate,Supine,"[Strength,, Stretch,, Balance]",NaN,Wind Release Pose,Supine Hip Flexor Roller,Supine Hip Flexor Roller
3636,Supine Knee Rolls Straps,Beginner,Supine,[Stretch],NaN,Wind Release Pose,Supine Knee Rolls Straps,Supine Knee Rolls Straps
3637,Supine Leg Lifts With Baby,Beginner,Supine,[Strength],NaN,Wind Release Pose,Supine Leg Lifts With Baby,Supine Leg Lifts With Baby


In [90]:
yoga_poses_df = yoga_poses_df[["Pose Name", "Level", "Position", "Movement Type", "Chakras", "Base Pose"]]

In [91]:
yoga_poses_df

,Pose Name,Level,Position,Movement Type,Chakras,Base Pose
0,Complete Boat Pose,Intermediate,Sitting,"[Forward-Bend,, Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Complete Boat Pose Contraindications
1,Boat Pose Aerial Variation,Intermediate,Sitting,"[Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Navasana
2,Boat Pose Aerial,Advanced,Sitting,"[Balance,, Strength]",[Solar Plexus (Manipura Chakra)],Boat Pose
3,Boat Pose Variation 1 Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
4,Boat Pose Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch]",[Solar Plexus (Manipura Chakra)],Boat Pose
...,...,...,...,...,...,...
3634,Wind Release Pose To Standing Pose Flow,Beginner,Supine,[Strength],[Solar Plexus (Manipura Chakra)],Wind Release Pose
3635,Supine Hip Flexor Roller,Intermediate,Supine,"[Strength,, Stretch,, Balance]",NaN,Wind Release Pose
3636,Supine Knee Rolls Straps,Beginner,Supine,[Stretch],NaN,Wind Release Pose
3637,Supine Leg Lifts With Baby,Beginner,Supine,[Strength],NaN,Wind Release Pose


Now, checking out the base poses, since I was a little concerned about it finding these correctly as it was just embedded in a text:

In [92]:
yoga_poses_df["Base Pose"].unique()

array(['Complete Boat Pose Contraindications', 'Navasana', 'Boat Pose',
       'request it here', 'Balancing Stick Pose Contraindications',
       'blanket', 'Supine Forearm Scissors Flow Contraindications',
       'pregnant women', 'Baddha Konasana Contraindications',
       'Bound Angle Pose', 'Baddha Konasana', nan,
       'Baddha Konasana Uttanasana Contraindications',
       'Upavistha Konasana', 'Bow Pose Contraindications', 'Bow Pose',
       'Cervical Spondylosis', 'menstruation',
       'Bridge Pose Contraindications', 'Bridge Pose',
       'Bridge Pose Flow Bound Angle Contraindications',
       'Chair Hip Opening And Strength Flow Contraindications',
       'Bridge Pose Flow Contraindications',
       'One Legged Bridge Pose Blocks Under Foot Leg Flow Contraindications',
       'Camel Pose Contraindications', 'Camel Pose', 'blood pressure',
       'Kapotasana Contraindications',
       'Bitilasana Marjaryasana Contraindications', 'Cat Cow Pose',
       'Bitilasana Vinyasa Co

Yeah, given the above, it did not find the right information for base poses, which is important for analysis. Dropping those, and running again to grab correct base poses: 

In [93]:
del yoga_poses_df["Base Pose"]

In [78]:
base_pose = soup.find("main").find(class_="pose-section").find_next(class_="pose-section").find_next(class_="pose-section").find("a").text

In [79]:
base_pose # wrong

'back pain'

In [83]:
soup.find(class_="pose", text="Variations").find_next("a").text # right! 

'Thread The Needle Pose'

In [97]:
def get_base_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    try: 
        english_name = soup.find(class_="pose-summary-header", text="English").find_next("td").text
        base_pose = soup.find(class_="pose", text="Variations").find_next("a").text
    except AttributeError:
        info = {"Pose": np.nan}
    except TimeoutError:
        info = {"Pose": np.nan}
    else: 
        info = {"Pose Name": english_name, "Base Pose": base_pose}
    return info

In [98]:
base_poses = [get_base_poses(url) for url in pose_links]

In [100]:
base_pose_df = pd.DataFrame(base_poses)

In [101]:
base_pose_df

,Pose Name,Base Pose,Pose
0,Complete Boat Pose,Boat Pose,NaN
1,Boat Pose Aerial Variation,Boat Pose,NaN
2,Boat Pose Aerial,Boat Pose,NaN
3,Boat Pose Variation 1 Arms Overhead,Boat Pose,NaN
4,Boat Pose Arms Overhead,Boat Pose,NaN
...,...,...,...
3634,Wind Release Pose To Standing Pose Flow,Wind Release Pose,NaN
3635,Supine Hip Flexor Roller,Wind Release Pose,NaN
3636,Supine Knee Rolls Straps,Wind Release Pose,NaN
3637,Supine Leg Lifts With Baby,Wind Release Pose,NaN


In [102]:
base_pose_df["Base Pose"].unique() # much better! these are actually the fundamental yoga poses!

array(['Boat Pose', 'request it here', 'Bound Angle Pose', 'Bow Pose',
       'Bridge Pose', 'Camel Pose', 'Cat Cow Pose',
       'Cat Tail Pulling Pose', 'Center Splits Pose', 'Chair Pose',
       'Child Pose', 'Cobra Pose', 'Corpse Pose', 'Savasana',
       'Cow Face Pose', 'Crane Pose', 'Crow Pose', 'Cycling Pose',
       'Dancer Pose', 'Dolphin Pose', 'Downward Facing Dog Pose',
       'Eagle Pose', 'Easy Pose', 'Easy Pose Chair', 'Exercise',
       'Extended Side Angle Pose', 'Eye Exercise',
       'Eye Of The Needle Pose', 'Feathered Peacock Pose', 'Firelog Pose',
       'Fish Pose', 'Flamingo Pose', 'Foot Behind The Head Pose',
       'Four Limbed Staff Pose', 'Garland Pose', 'Gate Pose',
       'Goddess Pose', 'Half Lord Of The Fishes Pose', 'Half Moon Pose',
       'Handstand Pose', 'Happy Baby Pose', 'Head To Knee Pose',
       'Headstand Pose', 'Hero Pose', nan, 'High Lunge Pose',
       'Intense Leg Stretch Pose', 'Intense Side Stretch Pose',
       'Legs Up The Wall Pose',

In [106]:
yoga_poses_df = pd.concat([yoga_poses_df, base_pose_df], axis=1)

In [107]:
yoga_poses_df

,Pose Name,Level,Position,Movement Type,Chakras,Pose Name,Base Pose,Pose
0,Complete Boat Pose,Intermediate,Sitting,"[Forward-Bend,, Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Complete Boat Pose,Boat Pose,NaN
1,Boat Pose Aerial Variation,Intermediate,Sitting,"[Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose Aerial Variation,Boat Pose,NaN
2,Boat Pose Aerial,Advanced,Sitting,"[Balance,, Strength]",[Solar Plexus (Manipura Chakra)],Boat Pose Aerial,Boat Pose,NaN
3,Boat Pose Variation 1 Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose Variation 1 Arms Overhead,Boat Pose,NaN
4,Boat Pose Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch]",[Solar Plexus (Manipura Chakra)],Boat Pose Arms Overhead,Boat Pose,NaN
...,...,...,...,...,...,...,...,...
3634,Wind Release Pose To Standing Pose Flow,Beginner,Supine,[Strength],[Solar Plexus (Manipura Chakra)],Wind Release Pose To Standing Pose Flow,Wind Release Pose,NaN
3635,Supine Hip Flexor Roller,Intermediate,Supine,"[Strength,, Stretch,, Balance]",NaN,Supine Hip Flexor Roller,Wind Release Pose,NaN
3636,Supine Knee Rolls Straps,Beginner,Supine,[Stretch],NaN,Supine Knee Rolls Straps,Wind Release Pose,NaN
3637,Supine Leg Lifts With Baby,Beginner,Supine,[Strength],NaN,Supine Leg Lifts With Baby,Wind Release Pose,NaN


Now dropping duplicate/unnecessary columns:

In [108]:
yoga_poses_df = yoga_poses_df[["Pose Name", "Level", "Position", "Movement Type", "Chakras", "Base Pose"]]

In [109]:
yoga_poses_df

,Pose Name,Pose Name,Level,Position,Movement Type,Chakras,Base Pose
0,Complete Boat Pose,Complete Boat Pose,Intermediate,Sitting,"[Forward-Bend,, Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
1,Boat Pose Aerial Variation,Boat Pose Aerial Variation,Intermediate,Sitting,"[Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
2,Boat Pose Aerial,Boat Pose Aerial,Advanced,Sitting,"[Balance,, Strength]",[Solar Plexus (Manipura Chakra)],Boat Pose
3,Boat Pose Variation 1 Arms Overhead,Boat Pose Variation 1 Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
4,Boat Pose Arms Overhead,Boat Pose Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch]",[Solar Plexus (Manipura Chakra)],Boat Pose
...,...,...,...,...,...,...,...
3634,Wind Release Pose To Standing Pose Flow,Wind Release Pose To Standing Pose Flow,Beginner,Supine,[Strength],[Solar Plexus (Manipura Chakra)],Wind Release Pose
3635,Supine Hip Flexor Roller,Supine Hip Flexor Roller,Intermediate,Supine,"[Strength,, Stretch,, Balance]",NaN,Wind Release Pose
3636,Supine Knee Rolls Straps,Supine Knee Rolls Straps,Beginner,Supine,[Stretch],NaN,Wind Release Pose
3637,Supine Leg Lifts With Baby,Supine Leg Lifts With Baby,Beginner,Supine,[Strength],NaN,Wind Release Pose


And pickling for later use: 

In [110]:
f = open("yoga_poses_df", "wb")
pickle.dump(yoga_poses_df, f)
f.close()

In [111]:
f = open("yoga_poses_df", "rb")
yoga_poses_df = pickle.load(f)
f.close()

In [112]:
yoga_poses_df

,Pose Name,Pose Name,Level,Position,Movement Type,Chakras,Base Pose
0,Complete Boat Pose,Complete Boat Pose,Intermediate,Sitting,"[Forward-Bend,, Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
1,Boat Pose Aerial Variation,Boat Pose Aerial Variation,Intermediate,Sitting,"[Strength,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
2,Boat Pose Aerial,Boat Pose Aerial,Advanced,Sitting,"[Balance,, Strength]",[Solar Plexus (Manipura Chakra)],Boat Pose
3,Boat Pose Variation 1 Arms Overhead,Boat Pose Variation 1 Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch,, Balance]",[Solar Plexus (Manipura Chakra)],Boat Pose
4,Boat Pose Arms Overhead,Boat Pose Arms Overhead,Intermediate,Sitting,"[Strength,, Stretch]",[Solar Plexus (Manipura Chakra)],Boat Pose
...,...,...,...,...,...,...,...
3634,Wind Release Pose To Standing Pose Flow,Wind Release Pose To Standing Pose Flow,Beginner,Supine,[Strength],[Solar Plexus (Manipura Chakra)],Wind Release Pose
3635,Supine Hip Flexor Roller,Supine Hip Flexor Roller,Intermediate,Supine,"[Strength,, Stretch,, Balance]",NaN,Wind Release Pose
3636,Supine Knee Rolls Straps,Supine Knee Rolls Straps,Beginner,Supine,[Stretch],NaN,Wind Release Pose
3637,Supine Leg Lifts With Baby,Supine Leg Lifts With Baby,Beginner,Supine,[Strength],NaN,Wind Release Pose
